In [1]:
import sqlite3
import pandas as pd

DB1, DB2 = 'q4', 'fhir_test_01'

In [2]:
db1_conn = sqlite3.connect(f'{DB1}.db')

query = f"""
select project_local_id, count(local_id) as {DB1}_subject_count
from subject
group by project_local_id
"""
db1_subject_df = pd.read_sql_query(query,db1_conn)

In [3]:
db2_conn = sqlite3.connect(f'{DB2}.db')

query = f"""
select project_local_id, count(local_id) as {DB2}_subject_count
from subject
group by project_local_id
"""
db2_subject_df = pd.read_sql_query(query,db2_conn)

In [4]:
combined_quarters_df = db1_subject_df.merge(db2_subject_df,
                                   how='inner',
                                   on='project_local_id') \
                            .sort_values(by='project_local_id')

In [5]:
query = f"""
select project_local_id, count(local_id) as {DB1}_biosample_count
from biosample
group by project_local_id
"""
db1_biosample_df = pd.read_sql_query(query,db1_conn)

In [6]:
query = f"""
select project_local_id, count(local_id) as {DB2}_biosample_count
from biosample
group by project_local_id
"""
db2_biosample_df = pd.read_sql_query(query,db2_conn)

In [7]:
combined_quarters_df = combined_quarters_df.merge(db1_biosample_df,
                                                  how='inner',
                                                  on='project_local_id') \
                                            .merge(db2_biosample_df,
                                                   how='inner',
                                                   on='project_local_id')


In [8]:
query = f"""
select project_local_id, count(local_id) as {DB1}_file_count
from file
group by project_local_id
"""
db1_file_df = pd.read_sql_query(query,db1_conn)

In [9]:
query = f"""
select project_local_id, count(local_id) as {DB2}_file_count
from file
group by project_local_id
"""
db2_file_df = pd.read_sql_query(query,db2_conn)


In [10]:
combined_quarters_df = combined_quarters_df.merge(db1_file_df,
                                                  how='inner',
                                                  on='project_local_id') \
                                            .merge(db2_file_df,
                                                   how='inner',
                                                   on='project_local_id')



In [11]:
combined_quarters_df['subject_count_change'] = combined_quarters_df.apply(lambda row: row[f'{DB2}_subject_count'] - row[f'{DB1}_subject_count'],axis=1)
combined_quarters_df['biosample_count_change'] = combined_quarters_df.apply(lambda row: row[f'{DB2}_biosample_count'] - row[f'{DB1}_biosample_count'],axis=1)
combined_quarters_df['file_count_change'] = combined_quarters_df.apply(lambda row: row[f'{DB2}_file_count'] - row[f'{DB1}_file_count'],axis=1)

# combined_quarters_df.sort_values(by=['biosample_count_change'],inplace=True)
# count_change_df = combined_quarters_df[['project_local_id','subject_count_change','biosample_count_change','file_count_change']]
# count_change_df = count_change_df.query('subject_count_change != 0 and biosample_count_change != 0 and file_count_change != 0')
# count_change_df

combined_quarters_df

,project_local_id,q4_subject_count,fhir_test_01_subject_count,q4_biosample_count,fhir_test_01_biosample_count,q4_file_count,fhir_test_01_file_count,subject_count_change,biosample_count_change,file_count_change
0,SD_0TYVY1TW,334,334,334,668,1494,748,0,334,-746
1,SD_1P41Z782,137,137,259,518,259,259,0,259,0
2,SD_2CEKQ05V,255,255,255,510,1238,652,0,255,-586
3,SD_46RR9ZR6,341,341,1220,2440,2208,856,0,1220,-1352
4,SD_46SK55A3,2031,2031,2121,4242,10213,7351,0,2121,-2862
5,SD_54G4WG4R,517,517,517,1034,2449,1288,0,517,-1161
6,SD_6FPYJQBR,183,183,183,366,1021,1025,0,183,4
7,SD_7NQ9151J,211,211,217,434,1441,44,0,217,-1397
8,SD_8Y99QZJJ,77,77,365,466,18807,5066,0,101,-13741
9,SD_9PYZAHHE,1414,1414,1295,2590,7227,4656,0,1295,-2571
